In [13]:

import sys, os
sys.path.append(os.getcwd())
sys.path.append(os.path.abspath('..'))
from imports import ilpgraph, graph_parser
from network import PCST, PCST_Linear
import networkx as nx
from gurobipy import *
import xlsxwriter

def __write_solution(solution:list):
    
    workbook = xlsxwriter.Workbook('Benchmarks4.xlsx')
    worksheet = workbook.add_worksheet()
    bold = workbook.add_format({'bold':True})
    solutionRows = len(solution)
    solutionColumns = len(solution[0])
    
    worksheet.write('A1', 'Instance', bold)
    worksheet.write('B1', 'Edges', bold)
    worksheet.write('C1', 'Nodes', bold)
    worksheet.write('D1', 'LinearModel', bold)
    worksheet.write('D1', 'cycleBasis', bold)
    worksheet.write('E1', 'nodeColouring', bold)
    worksheet.write('F1', 'Runtime', bold)
    worksheet.write('G1', 'Objective Value', bold)
    worksheet.write('H1', 'Constraints', bold)
    worksheet.write('I1', 'MIP Gap', bold)
    
    for i in range(solutionRows):
        for j in range(solutionColumns):
            worksheet.write(i + 1, j, solution[i][j])
    workbook.close()
    print("Finished!")
def runBenchmark(settings:list, path:str, linesToRead:range):
    solutions = []
    lines = []
    with open(path) as f:
        formattedFile = f.read().split('\n')
        for i in linesToRead:
            lines.append(formattedFile[i])

    for i in range(len(linesToRead)):
        G, start_terminal, cost_per_meter, prob_id = graph_parser.parse_vdf(lines[i])
        Graph = ilpgraph.ILPGraph()
        Graph.setNXGraph(G)
        Graph.setEdgeVars(G.edges())
        Graph.setNodeVars(G.nodes())
        
        for setting in settings:
            takeLinear = setting[0]
            cycleBasis = setting[1]
            nodeColoring = setting[2]
            # Normal version is to be chosen
            if (takeLinear == False):
                model = PCST.createModel(Graph, start_terminal, cycleBasis, nodeColoring)
                model.params.MIPGap = 0.05
                model.params.TimeLimit = 600
                model.optimize(PCST.callback_cycle) 
            else:
                model = PCST_Linear.createModel(Graph, start_terminal,  cycleBasis, nodeColoring)
                model.params.MIPGap = 0.05
                model.params.TimeLimit = 600
                if (cycleBasis == True):
                    model.optimize(PCST.callback_cycle) 
                else:
                    model.optimize()
            
            
            solutions.append((i, len(G.edges()), len(G.nodes()), takeLinear, cycleBasis, nodeColoring, model.runTime, model.objVal, model.numConstrs, model.MIPGap))
    __write_solution(solutions)         

if __name__ == '__main__':
    linesToRead = range(0,3)
    
    settings = [(False, True, False), (True, True, False), (True, True, False), (False, True, True), (True, False, True), (True, True, True),
                (False, False, False), (False, False, True)]
    path = 'H:\optimisation_instances_02_he_new.txt'
    
    runBenchmark(settings, path, linesToRead)
    




2635.7048      -     -    0s
     0     2 12635.7048    0   81   -0.00000 12635.7048      -     -    0s
H  911   405                     300.1684779 11023.9121  3573%  36.6    2s
  1846   756 5224.30552   14   84  300.16848 9507.17240  3067%  34.3    5s
  3343   782 9507.17240   32   82  300.16848 9507.17240  3067%  43.5   10s
  5633  1153 9251.57776   24   34  300.16848 9507.17240  3067%  47.4   15s
  8591  2227 5434.37611   31   22  300.16848 9507.17240  3067%  44.3   20s
 12137  3408 9131.17747   27   31  300.16848 9467.66284  3054%  41.8   25s
 15596  4801 5531.86307   27   28  300.16848 9066.83452  2921%  40.2   30s
 17877  5433 5122.70665   30   81  300.16848 8879.60156  2858%  40.1   41s
 17898  5451 8879.60156   27  160  300.16848 8879.60156  2858%  40.3   45s
 18629  5493 infeasible   38       300.16848 8658.17468  2784%  40.5   50s
 21000  5749 5320.90585   38   34  300.16848 8181.07550  2625%  39.8   55s
 23706  5938 6660.09340   43   40  300.16848 7790.03194  2495%  39.3   

# Testing Steiner Module

In [14]:
import itertools
import sys
sys.path.append("../..")
from graphilp.imports import ilpgraph,read_stp
from graphilp.network import Steiner
import networkx as nx
from gurobipy import *


def runBenchmark(settings:list, path:str):
    solutions = []
    lines = []


    G, terminals = read_stp.stp_to_networkx(path)

    for setting in settings:
        cycleBasis = setting[0]
        nodeColoring = setting[1]
        model = Steiner.createModel(G, terminals, cycleBasis, nodeColoring)
        model.params.MIPGap = 0.02
        model.params.TimeLimit = 600
        if (cycleBasis):
            model.optimize(Steiner.callback_cycle) 
        else:
            model.optimize()
        solutions.append((len(G.edges()), len(G.nodes()), cycleBasis, nodeColoring, model.runTime, model.objVal,\
                          model.numConstrs, model.MIPGap))
        
    #__write_solution(solutions)         
    print(solutions)



if __name__ == '__main__':
    
    settings = [(False, False), (True,False), (False, True), (True, True)]
    path = '/mnt/data/projects/ILP/steiner/PUC/bip42u.stp'
    
    runBenchmark(settings, path)


ModuleNotFoundError: No module named 'graphilp'

In [7]:
import itertools
import sys
sys.path.append("../..")
from graphilp.imports import ilpgraph,read_stp
from graphilp.network import Steiner_Linear
import networkx as nx
from gurobipy import *


def runBenchmark(settings:list, path:str):
    solutions = []
    lines = []


    G, terminals = read_stp.stp_to_networkx(path)
    model = Steiner_Linear.createModel(G, terminals)
    model.params.MIPGap = 0.02
    model.params.TimeLimit = 600
    model.optimize()
    solutions.append((len(G.edges()), len(G.nodes()), model.runTime, model.objVal,\
                      model.numConstrs, model.MIPGap))
        
    #__write_solution(solutions)         
    print(solutions)



if __name__ == '__main__':
    
    path = '/mnt/data/projects/ILP/steiner/PUC/bip42u.stp'
    runBenchmark(settings, path)


Changed value of parameter threads to 3
   Prev: 0  Min: 0  Max: 1024  Default: 0
Changed value of parameter MIPGap to 0.02
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 3 threads
Optimize a model with 18529 rows, 10364 columns and 90204 nonzeros
Model fingerprint: 0xaae93009
Variable types: 0 continuous, 10364 integer (9164 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+03]
  RHS range        [1e+00, 1e+03]
Presolve removed 4278 rows and 248 columns
Presolve time: 0.10s
Presolved: 14251 rows, 10116 columns, 70262 nonzeros
Variable types: 0 continuous, 10116 integer (8916 binary)

Root relaxation: objective 2.318806e+02, 13664 iterations, 1.55 seconds

    Nodes    |    Current Node  

NameError: name 'cycleBasis' is not defined